In [1]:
import { MinPriorityQueue, MaxPriorityQueue } from '@datastructures-js/priority-queue';

In [2]:
// Explicit Max-Plus Algebra Implementation
// Using regular algebra with explicit max and plus operations

// Constants for epsilon (negative infinity) and infinity
const EPS = -Infinity;
const INF = Infinity;

// Type definitions
type Matrix = number[][];
type Vector = number[];

// Helper functions for explicit max-plus operations
function maxPlus(a: number, b: number): number {
  return Math.max(a, b);
}

function timesPlus(a: number, b: number): number {
  if (a === EPS || b === EPS) return EPS;
  if (a === INF || b === INF) return INF;
  return a + b;
}

// Matrix operations using explicit max and plus
function matrixMaxPlus(A: Matrix, B: Matrix): Matrix {
  const rows = A.length;
  const cols = B[0].length;
  const result: Matrix = [];
  
  for (let i = 0; i < rows; i++) {
    result[i] = [];
    for (let j = 0; j < cols; j++) {
      result[i][j] = EPS;
      for (let k = 0; k < A[i].length; k++) {
        result[i][j] = maxPlus(result[i][j], timesPlus(A[i][k], B[k][j]));
      }
    }
  }
  return result;
}

function matrixAdd(A: Matrix, B: Matrix): Matrix {
  const result: Matrix = [];
  for (let i = 0; i < A.length; i++) {
    result[i] = [];
    for (let j = 0; j < A[i].length; j++) {
      result[i][j] = maxPlus(A[i][j], B[i][j]);
    }
  }
  return result;
}

function matrixTranspose(A: Matrix): Matrix {
  const result: Matrix = [];
  for (let j = 0; j < A[0].length; j++) {
    result[j] = [];
    for (let i = 0; i < A.length; i++) {
      result[j][i] = A[i][j];
    }
  }
  return result;
}

console.log("Explicit max-plus algebra functions defined");


Explicit max-plus algebra functions defined


### Delay Analysis with Explicit Max-Plus Equations
Note: the actual delay tested might not be 100 ms.

<img src="../../img/Delay100ms.svg" alt="drawing" width="500"/>


In [3]:
// System Setup with Explicit Max-Plus Encoding

// Create state vector for system (earliest firing times)
let x: Matrix = [[EPS, EPS]];

// Indices for system components
const a = 0;  // Reaction A index
const b = 1;  // Reaction B index

console.log("System setup complete");
console.log("Initial state x =", x);


System setup complete
Initial state x = [ [ -Infinity, -Infinity ] ]


In [ ]:
// System Parameters

// Logical time
let t: number = EPS;
let tBar: Matrix = [[t, t]];

// Timer parameters
const offset = 0;
const period = 1; // A period of 0.15 (150ms) is too small, barrier sync constraints will dominate
let timerValue: number = offset;

// Delay parameter
const delay = 0.5; // A delay < 200 ms should have no effect on firing time because of barrier sync

// Maxwait for decentralized coordination
const maxwaitA = INF;
const maxwaitB = INF;

// Execution times
let e: Matrix = [[0.1, 0.1]];
const wcetA = 0.1;
const wcetB = 0.1;

// Identity matrix (explicit)
const Identity: Matrix = [
  [0, EPS],
  [EPS, 0],
];

// System matrices
let Gamma: Matrix;      // Dependency matrix
let GammaStar: Matrix;  // Kleene star of Gamma
let B: Matrix;          // Barrier synchronization matrix

console.log("System parameters initialized");


System parameters initialized


In [5]:
// For events sorted by time (earliest first)
interface Event {
  time: number;
  trigger: string;
}

const eventQ: MinPriorityQueue<Event> = new MinPriorityQueue<Event>((event) => event.time);

In [6]:
// System Update Functions with Explicit Max-Plus Operations

// function updateLogicalTime(kPrime: number, eventQ: MinPriorityQueue<Event>): void {
//   // Pop earliest event from the queue.
//   const e = eventQ.dequeue();
//   t = e.time;
//   // Update tBar.
//   tBar = [[t, t]];
// }

function updateApparentLatency(kPrime: number): void {

  // Update dependency matrix using explicit values
  // Gamma[i][j] represents dependency from reaction j to reaction i
  Gamma = [
    [0, EPS],        // Reaction A: depends on itself (0), not on B (EPS)
    [e[0][a], 0],    // Reaction B: depends on A with execution time e[a], on itself (0)
  ];
  
  // Compute GammaStar explicitly using Kleene star formula
  // For 2x2 matrix, we use: A* = I ⊕ A (sufficient condition: no positive circuits)
  // This is equivalent to: max(Identity[i][j], Gamma[i][j]) for each element
  GammaStar = [
    [maxPlus(Identity[0][0], Gamma[0][0]), maxPlus(Identity[0][1], Gamma[0][1])],
    [maxPlus(Identity[1][0], Gamma[1][0]), maxPlus(Identity[1][1], Gamma[1][1])],
  ];

  // Update barrier synchronization matrix
  // B[i][j] = execution time of reaction j (same for all rows due to barrier sync)
  B = [
    [e[0][a], e[0][b]],
    [e[0][a], e[0][b]],
  ];

  console.log("k'=", kPrime, " k=", kPrime-1);
  console.log("e:", e);
  console.log("Gamma:", Gamma);
  console.log("GammaStar:", GammaStar);
  console.log("B:", B);
}

// console.log("System update functions defined");


In [7]:
// System Evolution with Fully Explicit Max-Plus Equations
// Evolution equation: x' = (GammaStar ⊗ B) ⊗ x ⊕ (GammaStar ⊗ tBar)
// Written out row by row without intermediate matrix operations

function step(x: Matrix, tBar: Matrix): Matrix {
  const x_a = x[0][a];
  const x_b = x[0][b];
  const t_a = tBar[0][a];
  const t_b = tBar[0][b];
  
  console.log(`Current state: x_a=${x_a}, x_b=${x_b}, t_a=${t_a}, t_b=${t_b}`);
  
  // Explicitly compute each element of (GammaStar ⊗ B) ⊗ x ⊕ (GammaStar ⊗ tBar)
  // For reaction A (row 0):
  // x_a' = max over all paths: (GammaStar[0][j] + B[j][i] + x[i]) ⊕ (GammaStar[0][j] + tBar[j])
  
  const x_a_next =
  Math.min(maxwaitA + t_a,
  Math.max(
    // From (GammaStar ⊗ B) ⊗ x:
    timesPlus(timesPlus(GammaStar[0][0], B[0][0]), x_a),  // GammaStar[0][0] + B[0][0] + x_a
    timesPlus(timesPlus(GammaStar[0][0], B[0][1]), x_b),  // GammaStar[0][0] + B[0][1] + x_b
    timesPlus(timesPlus(GammaStar[0][1], B[1][0]), x_a),  // GammaStar[0][1] + B[1][0] + x_a
    timesPlus(timesPlus(GammaStar[0][1], B[1][1]), x_b),  // GammaStar[0][1] + B[1][1] + x_b
    // From (GammaStar ⊗ tBar):
    timesPlus(GammaStar[0][0], t_a),  // GammaStar[0][0] + t_a
    timesPlus(GammaStar[0][1], t_b)   // GammaStar[0][1] + t_b
  ));
  
  // For reaction B (row 1):
  const x_b_next = 
  Math.min(maxwaitB + t_b,
  Math.max(
    // From (GammaStar ⊗ B) ⊗ x:
    timesPlus(timesPlus(GammaStar[1][0], B[0][0]), x_a),  // GammaStar[1][0] + B[0][0] + x_a
    timesPlus(timesPlus(GammaStar[1][0], B[0][1]), x_b),  // GammaStar[1][0] + B[0][1] + x_b
    timesPlus(timesPlus(GammaStar[1][1], B[1][0]), x_a),  // GammaStar[1][1] + B[1][0] + x_a
    timesPlus(timesPlus(GammaStar[1][1], B[1][1]), x_b),  // GammaStar[1][1] + B[1][1] + x_b
    // From (GammaStar ⊗ tBar):
    timesPlus(GammaStar[1][0], t_a),  // GammaStar[1][0] + t_a
    timesPlus(GammaStar[1][1], t_b)   // GammaStar[1][1] + t_b
  ));
  
  console.log(`Equation for x_a':
    max(
      GammaStar[0][0] + B[0][0] + x_a = ${GammaStar[0][0]} + ${B[0][0]} + ${x_a} = ${timesPlus(timesPlus(GammaStar[0][0], B[0][0]), x_a)},
      GammaStar[0][0] + B[0][1] + x_b = ${GammaStar[0][0]} + ${B[0][1]} + ${x_b} = ${timesPlus(timesPlus(GammaStar[0][0], B[0][1]), x_b)},
      GammaStar[0][1] + B[1][0] + x_a = ${GammaStar[0][1]} + ${B[1][0]} + ${x_a} = ${timesPlus(timesPlus(GammaStar[0][1], B[1][0]), x_a)},
      GammaStar[0][1] + B[1][1] + x_b = ${GammaStar[0][1]} + ${B[1][1]} + ${x_b} = ${timesPlus(timesPlus(GammaStar[0][1], B[1][1]), x_b)},
      GammaStar[0][0] + t_a = ${GammaStar[0][0]} + ${t_a} = ${timesPlus(GammaStar[0][0], t_a)},
      GammaStar[0][1] + t_b = ${GammaStar[0][1]} + ${t_b} = ${timesPlus(GammaStar[0][1], t_b)}
    ) = ${x_a_next}`);
  
  console.log(`Equation for x_b':
    max(
      GammaStar[1][0] + B[0][0] + x_a = ${GammaStar[1][0]} + ${B[0][0]} + ${x_a} = ${timesPlus(timesPlus(GammaStar[1][0], B[0][0]), x_a)},
      GammaStar[1][0] + B[0][1] + x_b = ${GammaStar[1][0]} + ${B[0][1]} + ${x_b} = ${timesPlus(timesPlus(GammaStar[1][0], B[0][1]), x_b)},
      GammaStar[1][1] + B[1][0] + x_a = ${GammaStar[1][1]} + ${B[1][0]} + ${x_a} = ${timesPlus(timesPlus(GammaStar[1][1], B[1][0]), x_a)},
      GammaStar[1][1] + B[1][1] + x_b = ${GammaStar[1][1]} + ${B[1][1]} + ${x_b} = ${timesPlus(timesPlus(GammaStar[1][1], B[1][1]), x_b)},
      GammaStar[1][0] + t_a = ${GammaStar[1][0]} + ${t_a} = ${timesPlus(GammaStar[1][0], t_a)},
      GammaStar[1][1] + t_b = ${GammaStar[1][1]} + ${t_b} = ${timesPlus(GammaStar[1][1], t_b)}
    ) = ${x_b_next}`);
  
  return [[x_a_next, x_b_next]];
}

// Reporting function
function reportStats(k: number, x: Matrix, t: number): void {
  const lagK = x[0].map(xi => xi - t); // lag = earliest firing time - logical time

  console.log(`Tag index k=${k}`);
  console.log(`t(k)= ${t}`);
  console.log(`x(k) = earliest possible firing times = [${x[0].join(', ')}]`);
  console.log(`lag(k) = [${lagK.join(', ')}]`);
}  

console.log("System evolution functions defined");


System evolution functions defined


In [8]:
// System Simulation Setup

// Initialize logical time
t = 0;
tBar = [[t, t]];

// Reset state vector to epsilon (no reactions have fired yet)
x = [[EPS, EPS]];

// Iteration counter
let k = 0;

console.log("System simulation ready");
console.log("Initial state:", t, x, tBar);


System simulation ready
Initial state: 0 [ [ -Infinity, -Infinity ] ] [ [ 0, 0 ] ]


### Execute the simulation repeatedly

Run the following cell multiple times to step through the simulation.


In [9]:
// Simulation Execution - Run Multiple Iterations

let xArr: Matrix[] = [];
let tArr: number[] = [];
let eArr: number[][] = [];

// Add initial events
eventQ.enqueue({ time: 0, trigger: 'timer' });
eventQ.enqueue({ time: 0, trigger: 'inB' });

// Unroll execution for 10 iterations
for (let k = 0; k < 10; k++) {

  //// Update logical time and apparent latency for this iteration
  // Pop earliest event from the queue.
  t = eventQ.front() ? eventQ.front().time : t; // Earliest time
  let events: Event[] = [];
  while (!eventQ.isEmpty() && eventQ.front()!.time === t) {
    events.push(eventQ.dequeue());
  }
  // Update tBar.
  tBar = [[t, t]];

  // Update apparent latencies by inspecting popped off events.
  // Push new events into the queue based on popped off events.
  e = [[0, 0]]; // Initialize both reactions to be skipped.
  events.forEach(event => {
    switch (event.trigger) {
      case 'timer':
        // Mark reaction A as invoked.
        e[0][a] = wcetA;
        // The timer and inB triggers are semantically simultaneous
        // because there are zero delays from A to B, so enqueue both.
        eventQ.enqueue({ time: t + period, trigger: 'timer' });
        eventQ.enqueue({ time: t + period, trigger: 'inB' });
        break;
      case 'inA':
        // Mark reaction A as invoked.
        e[0][a] = wcetA;
        break;
      case 'inB':
        // Mark reaction B as invoked.
        e[0][b] = wcetB;
        // Enqueue downstream event after delay.
        eventQ.enqueue({ time: t + delay, trigger: 'inA' });
        break;
    }
  });
  eArr.push([...e[0]]); // Store execution times e(k)
  updateApparentLatency(k); // Update matrices.

  // Step the system from k to k+1 using explicit max-plus equations
  x = step(x, tBar);
  xArr.push(x.map(row => [...row])); // Store x(k') (deep copy)
  tArr.push(t); // Store t(k')
  
  console.log("=".repeat(50));
  console.log("Iteration k =", k);
  console.log("Logical time t =", t);
  console.log("State x =", x);
  console.log("Lag = [", x[0].map(xi => (xi - t).toFixed(3)).join(', '), "]");
}

console.log("\n" + "=".repeat(60));
console.log("SIMULATION COMPLETE");
console.log("Final results stored in xArr, tArr, eArr");


k'= 0  k= -1
e: [ [ 0.1, 0.1 ] ]
Gamma: [ [ 0, -Infinity ], [ 0.1, 0 ] ]
GammaStar: [ [ 0, -Infinity ], [ 0.1, 0 ] ]
B: [ [ 0.1, 0.1 ], [ 0.1, 0.1 ] ]
Current state: x_a=-Infinity, x_b=-Infinity, t_a=0, t_b=0
Equation for x_a':
    max(
      GammaStar[0][0] + B[0][0] + x_a = 0 + 0.1 + -Infinity = -Infinity,
      GammaStar[0][0] + B[0][1] + x_b = 0 + 0.1 + -Infinity = -Infinity,
      GammaStar[0][1] + B[1][0] + x_a = -Infinity + 0.1 + -Infinity = -Infinity,
      GammaStar[0][1] + B[1][1] + x_b = -Infinity + 0.1 + -Infinity = -Infinity,
      GammaStar[0][0] + t_a = 0 + 0 = 0,
      GammaStar[0][1] + t_b = -Infinity + 0 = -Infinity
    ) = 0
Equation for x_b':
    max(
      GammaStar[1][0] + B[0][0] + x_a = 0.1 + 0.1 + -Infinity = -Infinity,
      GammaStar[1][0] + B[0][1] + x_b = 0.1 + 0.1 + -Infinity = -Infinity,
      GammaStar[1][1] + B[1][0] + x_a = 0 + 0.1 + -Infinity = -Infinity,
      GammaStar[1][1] + B[1][1] + x_b = 0 + 0.1 + -Infinity = -Infinity,
      GammaStar[1][0] + 

## Timeline Visualization

Let's create functions to visualize the simulation results.


In [10]:
// Timeline Data Extraction (adapted for explicit implementation)
function extractTimelineData(xArr: Matrix[], tArr: number[] = []) {
  const reactions: { [key: string]: Array<{tagIndex: number, time: number, logicalTime: number, lag: number, execTime: number}> } = {};
  
  if (xArr.length === 0) return { reactionA: [], reactionB: [] };
  
  // Initialize reaction arrays based on matrix width
  const numReactions = xArr[0][0].length;
  const reactionNames = Array.from({ length: numReactions }, (_, i) => String.fromCharCode(65 + i)); // A, B, C, D, ...
  
  reactionNames.forEach(name => {
    reactions[name] = [];
  });
  
  xArr.forEach((matrix, k) => {
    const logicalTime = tArr.length > k ? tArr[k] : 0; // Get logical time for this iteration
    matrix[0].forEach((time, reactionIndex) => {
      if (time !== EPS && time !== INF) {
        const reactionName = reactionNames[reactionIndex];
        const lag = time - logicalTime; // Calculate lag as x - t
        const execTime = eArr.length > k ? eArr[k][reactionIndex] : 0; // Get execution time
        reactions[reactionName].push({ 
          tagIndex: k, 
          time: time, 
          logicalTime: logicalTime,
          lag: lag,
          execTime: execTime,
        });
      }
    });
  });
  
  // For backward compatibility, still return reactionA and reactionB
  return { 
    reactionA: reactions['A'] || [], 
    reactionB: reactions['B'] || [],
    ...reactions
  };
}

console.log("Timeline data extraction function defined");


Timeline data extraction function defined


### Generate HTML Timeline

Create a comprehensive HTML timeline with Vis.js integration.


In [11]:
// HTML Timeline Generation (same as original but adapted for explicit implementation)
function generateHTMLTimeline(xArr: Matrix[], tArr: number[] = []) {
  const reactionData = extractTimelineData(xArr, tArr);
  
  // Create a generic color palette for reactions
  const colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c', '#e67e22', '#34495e'];
  
  // Create Vis.js timeline items and groups dynamically
  const items: any[] = [];
  const groups: any[] = [];
  const allReactions: Array<{name: string, events: Array<{tagIndex: number, time: number, logicalTime: number, lag: number, execTime: number}>, color: string}> = [];
  
  // Build reaction list dynamically
  Object.keys(reactionData).forEach((key, index) => {
    // Skip the backward compatibility keys and only use the actual reaction keys
    if (key.length === 1 && reactionData[key].length > 0) {
      allReactions.push({
        name: key,
        events: reactionData[key],
        color: colors[index % colors.length]
      });
    }
  });
  
  // Create groups
  allReactions.forEach((reaction) => {
    groups.push({
      id: reaction.name,
      content: `Reaction ${reaction.name}`,
      style: `background-color: white; color: #2c3e50; border-left: 4px solid ${reaction.color};`
    });
  });
  
  // Create items
  allReactions.forEach((reaction) => {
    reaction.events.forEach(event => {
      // Create multi-line content with k + index, logical time, and lag
      const content = `k = ${event.tagIndex}<br/>x = ${event.time.toFixed(3)}<br/>t = ${event.logicalTime.toFixed(3)}<br/>lag = ${event.lag.toFixed(3)}<br/>e = ${event.execTime.toFixed(3)}`;
      
      items.push({
        id: `${reaction.name}_${event.tagIndex}`,
        content: content,
        start: event.time * 1000, // Use milliseconds from epoch as relative time
        group: reaction.name,
        title: `Tag ${event.tagIndex}: Reaction ${reaction.name} fires at ${event.time.toFixed(3)}s (logical time: ${event.logicalTime.toFixed(3)}s, lag: ${event.lag.toFixed(3)}s)`,
        style: `background-color: ${reaction.color}; border-color: ${reaction.color}; color: white;`
      });
    });
  });
  
  // Calculate statistics
  const totalEvents = items.length;
  
  // Calculate average delays between consecutive reactions if they exist
  let avgDelay = 0;
  if (allReactions.length >= 2 && xArr.length > 0) {
    const delays: number[] = [];
    xArr.forEach((matrix, k) => {
      for (let i = 0; i < matrix[0].length - 1; i++) {
        const time1 = matrix[0][i];
        const time2 = matrix[0][i + 1];
        if (time1 !== EPS && time1 !== INF && time2 !== EPS && time2 !== INF) {
          delays.push(Math.abs(time2 - time1));
        }
      }
    });
    avgDelay = delays.length > 0 ? delays.reduce((sum, d) => sum + d, 0) / delays.length : 0;
  }
  
  const htmlContent = `<!DOCTYPE HTML>
<html>
<head>
  <title>CAL Timeline - Explicit Max-Plus</title>
  <style type="text/css">
    body, html {
      font-family: sans-serif;
    }
    .header {
      background-color: #f8f9fa;
      padding: 20px;
      border-bottom: 2px solid #dee2e6;
      margin-bottom: 20px;
    }
    .header h1 {
      margin: 0;
      color: #2c3e50;
    }
    .header p {
      margin: 5px 0 0 0;
      color: #6c757d;
    }
  </style>
  <script src="https://unpkg.com/vis-timeline@latest/standalone/umd/vis-timeline-graph2d.min.js"></script>
  <link href="https://unpkg.com/vis-timeline@latest/styles/vis-timeline-graph2d.min.css" rel="stylesheet" type="text/css" />
</head>
<body>
<div class="header">
  <h1>Delay System Analysis - Explicit Max-Plus Implementation</h1>
  <p>Timeline showing reaction firing times using explicit max and plus operations instead of max-plus linear algebra</p>
</div>
<div id="visualization"></div>

<script type="text/javascript">
  // DOM element where the Timeline will be attached
  var container = document.getElementById('visualization');

  // Create a DataSet (allows two way data-binding)
  var items = new vis.DataSet(${JSON.stringify(items)});
  var groups = new vis.DataSet(${JSON.stringify(groups)});

  // Configuration for the Timeline
  var options = {
    format: {
      minorLabels: function(date, scale, step) {
        const seconds = (typeof date === 'number' ? date : new Date(date).getTime()) / 1000;
        return seconds.toFixed(1) + 's';
      },
      majorLabels: function(date, scale, step) {
        const seconds = (typeof date === 'number' ? date : new Date(date).getTime()) / 1000;
        return seconds.toFixed(2) + 's';
      }
    }
  };

  // Create a Timeline
  var timeline = new vis.Timeline(container, items, groups, options);
</script>
</body>
</html>`;

  return htmlContent;
}

console.log("HTML timeline generation function defined");


HTML timeline generation function defined


### Write HTML Timeline to File

Function to write the HTML timeline to a file on disk.


In [12]:
// Function to write HTML timeline to file (Node.js only)
function writeTimelineToFile(xArr: Matrix[], tArr: number[] = [], filename: string = 'reaction-timeline-explicit.html') {
  const htmlContent = generateHTMLTimeline(xArr, tArr);
  
  try {
    // Check if we're in a Node.js environment
    if (typeof require !== 'undefined') {
      const fs = require('fs');
      const path = require('path');
      
      // Write to current directory
      const fullPath = path.resolve(filename);
      fs.writeFileSync(fullPath, htmlContent, 'utf8');
      
      console.log(`✅ Timeline saved to: ${fullPath}`);
      console.log(`🌐 Open the file in your browser to view the interactive timeline`);
      console.log(`📊 This timeline shows results from explicit max-plus calculations`);
      
      return fullPath;
    } else {
      // Not in Node.js - just output the HTML content
      console.log(`⚠️  File writing not available in this environment`);
      console.log(`📋 Copy the HTML content below and save as '${filename}':`);
      console.log('='.repeat(60));
      console.log(htmlContent);
      console.log('='.repeat(60));
      return null;
    }
  } catch (error) {
    console.error('❌ Error writing timeline file:', error);
    console.log('\n📋 HTML Content (copy and save manually):');
    console.log('='.repeat(50));
    console.log(htmlContent);
    console.log('='.repeat(50));
    return null;
  }
}

// Alternative function to just get the HTML content
function getTimelineHTML(xArr: Matrix[], tArr: number[] = []): string {
  return generateHTMLTimeline(xArr, tArr);
}

// Create and save the timeline
console.log("🚀 Creating timeline with explicit max-plus calculations...");
const result = writeTimelineToFile(xArr, tArr, 'reaction-timeline-explicit.html');

// Also provide the HTML content for manual saving
console.log("\n💡 Alternative: Get HTML content directly:");
console.log("const htmlContent = getTimelineHTML(xArr, tArr);");


🚀 Creating timeline with explicit max-plus calculations...
✅ Timeline saved to: /Users/shaokai/Documents/projects/lf/cal-analyzer/notebooks/ts/reaction-timeline-explicit.html
🌐 Open the file in your browser to view the interactive timeline
📊 This timeline shows results from explicit max-plus calculations

💡 Alternative: Get HTML content directly:
const htmlContent = getTimelineHTML(xArr, tArr);


## Summary

This notebook demonstrates the same Delay system analysis as the original, but using **explicit max-plus algebra** instead of the max-plus linear algebra library.

### Key Differences:

1. **Explicit Operations**: Instead of using library functions like `oplus` and `otimes`, we use explicit `Math.max()` and `+` operations.

2. **Matrix Operations**: 
   - Max-plus matrix multiplication: `(A ⊗ B)[i,j] = max_k(A[i,k] + B[k,j])`
   - Max-plus addition: `(A ⊕ B)[i,j] = max(A[i,j], B[i,j])`

3. **Evolution Equation**: 
   ```
   x'(k+1) = (Γ* ⊗ B) ⊗ x(k) ⊕ (Γ* ⊗ t̄(k))
   ```
   Where:
   - `⊗` is max-plus matrix multiplication
   - `⊕` is max-plus addition (element-wise max)
   - `Γ*` is the Kleene star of the dependency matrix
   - `B` is the barrier synchronization matrix

4. **Explicit Calculations**: All operations are written out explicitly, making the max-plus algebra transparent and educational.

### Results:
The simulation should produce identical results to the original notebook, demonstrating that explicit max-plus operations yield the same system behavior as the library-based approach.
